In [ ]:
"""
Run Optuna Hyperparameter Optimization for ALL Lightning ML Models
===================================================================
Complete notebook-ready script with all functions included
"""

import optuna
import inspect
from sklearn.datasets import load_iris, load_diabetes, make_blobs
from sklearn.model_selection import train_test_split
from sklearn.metrics import silhouette_score, mean_squared_error, accuracy_score, r2_score

# Import all model classes
from lightning_ml.regression import (
    LinearRegression,
    LogisticRegression,
    RidgeRegression,
    LassoRegression
)

from lightning_ml.tree import (
    DecisionTreeClassifier,
    DecisionTreeRegressor
)

from lightning_ml.ensemble import (
    RandomForestClassifier,
    RandomForestRegressor,
    BaggingClassifier
)

from lightning_ml.svm import (
    SVMClassifier,
    SVMRegressor
)

from lightning_ml.neighbours import (
    KNNClassifier,
    KNNRegressor
)

from lightning_ml.cluster import (
    KMeans,
    DBSCAN
)

# Import suggest functions from your optuna_optimizer module
from lightning_ml.optuna_optimizer import (
    suggest_linear_regression_params,
    suggest_logistic_regression_params,
    suggest_ridge_regression_params,
    suggest_lasso_regression_params,
    suggest_svm_classifier_params,
    suggest_svm_regressor_params,
    suggest_decision_tree_classifier_params,
    suggest_decision_tree_regressor_params,
    suggest_random_forest_classifier_params,
    suggest_random_forest_regressor_params,
    suggest_bagging_classifier_params,
    suggest_knn_classifier_params,
    suggest_knn_regressor_params,
    suggest_kmeans_params,
    suggest_dbscan_params,
    suggest_apriori_params
)

# ============================================================================
# UTILITY FUNCTIONS
# ============================================================================

def filter_model_params(params, model_class):
    """Filter parameters to only include those accepted by the model"""
    sig = inspect.signature(model_class.__init__)
    valid_params = set(sig.parameters.keys()) - {'self'}
    filtered = {k: v for k, v in params.items() if k in valid_params}
    return filtered


def (model_class, suggest_params_func, X_train, y_train, 
                     X_test, y_test, metric='accuracy', n_trials=10, 
                     timeout=None, direction='maximize'):
    """Create and run Optuna optimization"""
    
    def objective(trial):
        params = suggest_params_func(trial)
        model_params = filter_model_params(params, model_class)
        
        model = model_class(**model_params)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        if metric == 'accuracy':
            score = accuracy_score(y_test, y_pred)
        elif metric == 'mse':
            score = -mean_squared_error(y_test, y_pred)
        elif metric == 'r2':
            score = r2_score(y_test, y_pred)
        else:
            raise ValueError(f"Unknown metric: {metric}")
        
        return score
    
    study = optuna.create_study(
        direction=direction,
        sampler=optuna.samplers.TPESampler(seed=42)
    )
    
    study.optimize(objective, n_trials=n_trials, timeout=timeout, show_progress_bar=True)
    
    best_params = filter_model_params(study.best_params, model_class)
    best_model = model_class(**best_params)
    best_model.fit(X_train, y_train)
    
    return best_model, study


def print_optimization_results(study, model_name):
    """Print optimization results"""
    print(f"\n{'='*70}")
    print(f"✅ {model_name} Optimization Complete!")
    print(f"{'='*70}")
    print(f"Best Score: {study.best_value:.4f}")
    print(f"\nBest Hyperparameters:")
    for key, value in study.best_params.items():
        print(f"  • {key}: {value}")
    print(f"\nTotal Trials: {len(study.trials)}")
    print(f"{'='*70}\n")


# =====================================================================
# MODEL GROUPS
# =====================================================================
regression_models = {
    "LinearRegression": (LinearRegression, suggest_linear_regression_params),
    "RidgeRegression": (RidgeRegression, suggest_ridge_regression_params),
    "LassoRegression": (LassoRegression, suggest_lasso_regression_params),
    "SVMRegressor": (SVMRegressor, suggest_svm_regressor_params),
    "DecisionTreeRegressor": (DecisionTreeRegressor, suggest_decision_tree_regressor_params),
    "RandomForestRegressor": (RandomForestRegressor, suggest_random_forest_regressor_params),
    "KNNRegressor": (KNNRegressor, suggest_knn_regressor_params),
}

classification_models = {
    "LogisticRegression": (LogisticRegression, suggest_logistic_regression_params),
    "SVMClassifier": (SVMClassifier, suggest_svm_classifier_params),
    "DecisionTreeClassifier": (DecisionTreeClassifier, suggest_decision_tree_classifier_params),
    "RandomForestClassifier": (RandomForestClassifier, suggest_random_forest_classifier_params),
    "BaggingClassifier": (BaggingClassifier, suggest_bagging_classifier_params),
    "KNNClassifier": (KNNClassifier, suggest_knn_classifier_params),
}

clustering_models = {
    "KMeans": (KMeans, suggest_kmeans_params),
    "DBSCAN": (DBSCAN, suggest_dbscan_params),
}

apriori_model = {
    "Apriori": (suggest_apriori_params)
}



# =====================================================================
# RUNNER FUNCTIONS
# =====================================================================

def run_all_regressors(n_trials=10, timeout=120):
    """Run hyperparameter tuning for all regression models"""
    print("\n" + "="*70)
    print("🔍 REGRESSION MODELS - HYPERPARAMETER TUNING")
    print("="*70)
    
    X, y = load_diabetes(return_X_y=True)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    results = {}
    for name, (cls, suggest_func) in regression_models.items():
        print(f"\n{'='*70}")
        print(f"🚀 Running: {name}")
        print(f"{'='*70}")
        try:
            best_model, study = create_optimizer(
                model_class=cls,
                suggest_params_func=suggest_func,
                X_train=X_train, y_train=y_train,
                X_test=X_test, y_test=y_test,
                metric='r2',
                n_trials=n_trials,
                timeout=timeout,
                direction='maximize'
            )
            print_optimization_results(study, name)
            
            # Test the best model
            y_pred = best_model.predict(X_test)
            test_r2 = r2_score(y_test, y_pred)
            test_mse = mean_squared_error(y_test, y_pred)
            print(f"📊 Test Set Performance:")
            print(f"  • R² Score: {test_r2:.4f}")
            print(f"  • MSE: {test_mse:.4f}")
            
            results[name] = {
                'model': best_model,
                'study': study,
                'best_params': study.best_params,
                'best_score': study.best_value,
                'test_r2': test_r2,
                'test_mse': test_mse
            }
        except Exception as e:
            print(f"❌ [ERROR] {name} failed: {e}")
            import traceback
            traceback.print_exc()
    
    return results


def run_all_classifiers(n_trials=10, timeout=120):
    """Run hyperparameter tuning for all classification models"""
    print("\n" + "="*70)
    print("🔍 CLASSIFICATION MODELS - HYPERPARAMETER TUNING")
    print("="*70)
    
    X, y = load_iris(return_X_y=True)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    results = {}
    for name, (cls, suggest_func) in classification_models.items():
        print(f"\n{'='*70}")
        print(f"🚀 Running: {name}")
        print(f"{'='*70}")
        try:
            best_model, study = create_optimizer(
                model_class=cls,
                suggest_params_func=suggest_func,
                X_train=X_train, y_train=y_train,
                X_test=X_test, y_test=y_test,
                metric='accuracy',
                n_trials=n_trials,
                timeout=timeout,
                direction='maximize'
            )
            print_optimization_results(study, name)
            
            # Test the best model
            y_pred = best_model.predict(X_test)
            test_accuracy = accuracy_score(y_test, y_pred)
            print(f"📊 Test Set Accuracy: {test_accuracy:.4f}")
            
            results[name] = {
                'model': best_model,
                'study': study,
                'best_params': study.best_params,
                'best_score': study.best_value,
                'test_accuracy': test_accuracy
            }
        except Exception as e:
            print(f"❌ [ERROR] {name} failed: {e}")
            import traceback
            traceback.print_exc()
    
    return results


def run_all_clustering(n_trials=10, timeout=120):
    """Run hyperparameter tuning for all clustering models"""
    print("\n" + "="*70)
    print("🔍 CLUSTERING MODELS - HYPERPARAMETER TUNING")
    print("="*70)
    
    X, y = make_blobs(n_samples=300, centers=3, n_features=2, random_state=42)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    results = {}
    for name, (cls, suggest_func) in clustering_models.items():
        print(f"\n{'='*70}")
        print(f"🚀 Running: {name}")
        print(f"{'='*70}")
        try:
            def objective(trial):
                params = suggest_func(trial)
                model_params = filter_model_params(params, cls)
                model = cls(**model_params)
                model.fit(X_train)
                labels = model.predict(X_test)
                
                # Check if we have at least 2 clusters
                if len(set(labels)) < 2:
                    return -1.0
                
                score = silhouette_score(X_test, labels)
                return score

            study = optuna.create_study(
                direction='maximize', 
                sampler=optuna.samplers.TPESampler(seed=42)
            )
            study.optimize(objective, n_trials=n_trials, timeout=timeout, show_progress_bar=True)

            best_params = filter_model_params(study.best_params, cls)
            best_model = cls(**best_params)
            best_model.fit(X_train)
            
            print_optimization_results(study, name)
            
            # Test the best model
            test_labels = best_model.predict(X_test)
            if len(set(test_labels)) >= 2:
                test_silhouette = silhouette_score(X_test, test_labels)
                print(f"📊 Test Set Silhouette Score: {test_silhouette:.4f}")
            else:
                test_silhouette = -1.0
                print(f"⚠️  Test Set: Only 1 cluster found")
            
            results[name] = {
                'model': best_model,
                'study': study,
                'best_params': study.best_params,
                'best_score': study.best_value,
                'test_silhouette': test_silhouette
            }
        except Exception as e:
            print(f"❌ [ERROR] {name} failed: {e}")
            import traceback
            traceback.print_exc()
    
    return results


# =====================================================================
# SUMMARY FUNCTION
# =====================================================================

def print_summary(reg_results, cls_results, clu_results):
    """Print a summary of all results"""
    print("\n" + "="*70)
    print("📋 HYPERPARAMETER TUNING SUMMARY")
    print("="*70)
    
    if reg_results:
        print("\n🔹 REGRESSION MODELS:")
        print("-" * 70)
        for name, result in reg_results.items():
            print(f"{name:30s} | R²: {result['test_r2']:6.4f} | MSE: {result['test_mse']:8.2f}")
    
    if cls_results:
        print("\n🔹 CLASSIFICATION MODELS:")
        print("-" * 70)
        for name, result in cls_results.items():
            print(f"{name:30s} | Accuracy: {result['test_accuracy']:6.4f}")
    
    if clu_results:
        print("\n🔹 CLUSTERING MODELS:")
        print("-" * 70)
        for name, result in clu_results.items():
            print(f"{name:30s} | Silhouette: {result['test_silhouette']:6.4f}")
    
    print("\n" + "="*70)


# =====================================================================
# MAIN EXECUTION
# =====================================================================

if __name__ == "__main__":
    print("\n" + "="*70)
    print("⚡ LIGHTNING ML - HYPERPARAMETER OPTIMIZATION")
    print("="*70)
    print("Running Optuna hyperparameter tuning for all models...")
    
    # Run all optimizations
    reg_results = run_all_regressors(n_trials=10, timeout=120)
    cls_results = run_all_classifiers(n_trials=10, timeout=120)
    clu_results = run_all_clustering(n_trials=10, timeout=120)
    
    # Print summary
    print_summary(reg_results, cls_results, clu_results)
    
    print("\n✅ ALL HYPERPARAMETER TUNING COMPLETE!")
    print("="*70)

[I 2025-10-07 19:37:22,950] A new study created in memory with name: no-name-8fbafed6-b214-47da-be04-af5e88490355



⚡ LIGHTNING ML - HYPERPARAMETER OPTIMIZATION
Running Optuna hyperparameter tuning for all models...

🔍 REGRESSION MODELS - HYPERPARAMETER TUNING

🚀 Running: LinearRegression


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2025-10-07 19:37:24,313] A new study created in memory with name: no-name-34899789-36ea-4dd0-bf1a-3b326461c43b


[I 2025-10-07 19:37:24,233] Trial 0 finished with value: -4.277644824214061 and parameters: {'fit_intercept': False}. Best is trial 0 with value: -4.277644824214061.
[I 2025-10-07 19:37:24,246] Trial 1 finished with value: 0.4526029062055733 and parameters: {'fit_intercept': True}. Best is trial 1 with value: 0.4526029062055733.
[I 2025-10-07 19:37:24,254] Trial 2 finished with value: 0.4526029062055733 and parameters: {'fit_intercept': True}. Best is trial 1 with value: 0.4526029062055733.
[I 2025-10-07 19:37:24,261] Trial 3 finished with value: -4.277644824214061 and parameters: {'fit_intercept': False}. Best is trial 1 with value: 0.4526029062055733.
[I 2025-10-07 19:37:24,267] Trial 4 finished with value: -4.277644824214061 and parameters: {'fit_intercept': False}. Best is trial 1 with value: 0.4526029062055733.
[I 2025-10-07 19:37:24,275] Trial 5 finished with value: -4.277644824214061 and parameters: {'fit_intercept': False}. Best is trial 1 with value: 0.4526029062055733.
[I 202

...

[I 2025-10-07 19:46:24,247] Trial 0 finished with value: 0.3492229946215037 and parameters: {'n_clusters': 9, 'max_iter': 1000, 'tol': 0.0008471801418819979, 'n_init': 14, 'init': 'k-means++'}. Best is trial 0 with value: 0.3492229946215037.
[I 2025-10-07 19:46:24,475] Trial 1 finished with value: 0.8598282581279164 and parameters: {'n_clusters': 3, 'max_iter': 900, 'tol': 0.00025378155082656634, 'n_init': 16, 'init': 'random'}. Best is trial 1 with value: 0.8598282581279164.
[I 2025-10-07 19:46:25,314] Trial 2 finished with value: 0.3399124758701325 and parameters: {'n_clusters': 17, 'max_iter': 300, 'tol': 5.337032762603957e-06, 'n_init': 7, 'init': 'random'}. Best is trial 1 with value: 0.8598282581279164.
[I 2025-10-07 19:46:25,607] Trial 3 finished with value: 0.2645567328569067 and parameters: {'n_clusters': 10, 'max_iter': 300, 'tol': 0.0002801635158716264, 'n_init': 7, 'init': 'random'}. Best is trial 1 with value: 0.8598282581279164.
[I 2025-10-07 19:46:26,582] Trial 4 finishe

[I 2025-10-07 19:46:31,152] A new study created in memory with name: no-name-c9c2568e-187f-41ef-85d2-f75b3534baf6


[I 2025-10-07 19:46:31,011] Trial 9 finished with value: 0.32709947546691487 and parameters: {'n_clusters': 13, 'max_iter': 1000, 'tol': 2.259279742015697e-06, 'n_init': 8, 'init': 'random'}. Best is trial 1 with value: 0.8598282581279164.

✅ KMeans Optimization Complete!
Best Score: 0.8598

Best Hyperparameters:
  • n_clusters: 3
  • max_iter: 900
  • tol: 0.00025378155082656634
  • n_init: 16
  • init: random

Total Trials: 10

📊 Test Set Silhouette Score: 0.8598

🚀 Running: DBSCAN


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2025-10-07 19:46:31,545] Trial 0 finished with value: 0.5381981195209474 and parameters: {'eps': 1.9352465823520764, 'min_samples': 20, 'metric': 'euclidean'}. Best is trial 0 with value: 0.5381981195209474.
[I 2025-10-07 19:46:31,751] Trial 1 finished with value: -0.42001988702941406 and parameters: {'eps': 0.864373149647393, 'min_samples': 3, 'metric': 'euclidean'}. Best is trial 0 with value: 0.5381981195209474.
[I 2025-10-07 19:46:31,838] Trial 2 finished with value: -1.0 and parameters: {'eps': 0.200864022049432, 'min_samples': 20, 'metric': 'euclidean'}. Best is trial 0 with value: 0.5381981195209474.
[I 2025-10-07 19:46:31,961] Trial 3 finished with value: -0.153518615631543 and parameters: {'eps': 0.9986820982818257, 'min_samples': 7, 'metric': 'euclidean'}. Best is trial 0 with value: 0.5381981195209474.
[I 2025-10-07 19:46:32,205] Trial 4 finished with value: 0.5329120807450787 and parameters: {'eps': 3.0980791841396598, 'min_samples': 4, 'metric': 'cosine'}. Best is trial

In [11]:
if __name__ == "__main__":
    import warnings
    warnings.filterwarnings("ignore")  # Clean output

    print("\n" + "="*70)
    print("⚡ LIGHTNING ML - APRIORI OPTUNA TEST")
    print("="*70)

    import optuna
    from lightning_ml import Apriori  # Your Apriori class
    from lightning_ml.optuna_optimizer import suggest_apriori_params  # Your Optuna parameter suggestion function
    import pandas as pd
    import numpy as np

    # Example transactional dataset
    data = {
        'Milk': [1, 0, 1, 1, 0, 1, 1, 0],
        'Bread': [1, 1, 1, 1, 0, 0, 1, 1],
        'Butter': [1, 0, 1, 0, 0, 1, 1, 0],
        'Eggs': [0, 1, 1, 1, 1, 0, 1, 1],
        'Cheese': [0, 0, 1, 1, 0, 1, 0, 0]
    }
    df = pd.DataFrame(data)

    def objective(trial):
        # Suggest hyperparameters using your function
        params = suggest_apriori_params(trial)
        model = Apriori(**params)
        model.fit(df)
        # Use avg lift of top 10 rules as objective
        return model.score(df)

    # Run Optuna study
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=10, timeout=60)

    print("\n✅ Optuna optimization completed!")
    print(f"Best parameters: {study.best_params}")
    print(f"Best score (avg lift): {study.best_value:.4f}\n")


[I 2025-10-07 20:26:35,790] A new study created in memory with name: no-name-1071f3a5-8669-457e-9141-237dca0a944b
[I 2025-10-07 20:26:35,794] Trial 0 finished with value: 0.0 and parameters: {'use_max_length': False, 'min_support': 0.3761512710010716, 'min_confidence': 0.5049574728808788, 'min_lift': 2.930724414633814}. Best is trial 0 with value: 0.0.
[I 2025-10-07 20:26:35,795] Trial 1 finished with value: 0.0 and parameters: {'use_max_length': False, 'min_support': 0.2961692239929821, 'min_confidence': 0.8137449152492808, 'min_lift': 4.414664410423578}. Best is trial 0 with value: 0.0.
[I 2025-10-07 20:26:35,798] Trial 2 finished with value: 0.0 and parameters: {'use_max_length': False, 'min_support': 0.47296720530573905, 'min_confidence': 0.5090553327247854, 'min_lift': 4.126752017259424}. Best is trial 0 with value: 0.0.
[I 2025-10-07 20:26:35,801] Trial 3 finished with value: 0.0 and parameters: {'use_max_length': True, 'max_length': 9, 'min_support': 0.026160278217937843, 'min_c


⚡ LIGHTNING ML - APRIORI OPTUNA TEST

✅ Optuna optimization completed!
Best parameters: {'use_max_length': False, 'min_support': 0.2519486041154769, 'min_confidence': 0.4322436828302182, 'min_lift': 1.4326634897921595}
Best score (avg lift): 1.5750

